### Youtube Sentence Adjust Word List Ratio

In [42]:
#!pip install modin[ray]

In [43]:
#import ray  # ayrı olarak çalıştırılmalı
#ray.init() 

In [44]:
#import modin.pandas as pd

In [45]:
import pandas as pd
import numpy as np
import re
import glob

In [46]:
folder_name = "Turkish"
word_num = 28

#### Calculate Sentence Word Ratio

In [47]:
df_adjust_word = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{folder_name.lower().capitalize()}/Deployment/Result/Level 1/Opus/Word_{word_num}.xlsx")
df_adjust_word

,word,frequency
0,bir,18835735
1,bu,11062659
2,ne,8025880
3,ve,7766036
4,için,5484109
5,mi,5362714
6,ben,4908913
7,de,4880315
8,çok,4852169
9,ama,4661966


In [48]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{folder_name.lower().capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence = df_youtube_sentence.head(5000)
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:00:06.680,00:00:07.340,sen kimsin peki,JR7nKXxUVtM
1,00:00:07.340,00:00:08.600,parolayı söyle,JR7nKXxUVtM
2,00:00:09.440,00:00:11.060,ya ben kim olacağım,JR7nKXxUVtM
3,00:00:11.060,00:00:11.860,keloğlan ım,JR7nKXxUVtM
4,00:00:13.380,00:00:13.880,ey,JR7nKXxUVtM
...,...,...,...,...
4995,00:03:26.780,00:03:28.160,böyle bile yiyebiliriz,cQjX9DzXTw0
4996,00:03:28.260,00:03:30.540,kıvama geldiğinde artık tamamdır,cQjX9DzXTw0
4997,00:03:30.540,00:03:34.840,böyle ağzınıza böyle çıtır çıtır şeker tadı da...,cQjX9DzXTw0
4998,00:03:34.840,00:03:37.060,şimdi bundan sonra artık çırpıcı ile devam ede...,cQjX9DzXTw0


In [49]:
def sentence_word_ratio(df_word, df_sentence, word_column_name, sentence_column_name, word_num): # df_word and df_sentence are dataframe, word_column_name and sentence_column_name are string, word_num is an integer
    word_set = set(df_word[f"{word_column_name}"].to_list())
    for i in range(len(df_sentence)):
        sentence = df_sentence.loc[i,f"{sentence_column_name}"]
        sent_word = re.findall(r"\w+", sentence, re.UNICODE)
        sent_word_set = set(sent_word)
        intersect_word = word_set.intersection(sent_word_set)
        different_word = sent_word_set.difference(word_set)
        df_sentence.loc[i,f"{word_num}_word_ratio"] = (len(intersect_word)/len(sent_word)+0.001)*100
        df_sentence.loc[i,"different_word"] = [different_word]
        df_sentence.loc[i,"intersect_word"] = [intersect_word]

    df_sentence.reset_index(inplace=True)    
    return df_sentence

In [50]:
df_sentence_ratio_result = sentence_word_ratio(df_adjust_word, df_youtube_sentence, "word", "sentence", 28)
df_sentence_ratio_result

,index,start_time,end_time,sentence,video_id,28_word_ratio,different_word,intersect_word
0,0,00:00:06.680,00:00:07.340,sen kimsin peki,JR7nKXxUVtM,33.433333,"{kimsin, peki}",{sen}
1,1,00:00:07.340,00:00:08.600,parolayı söyle,JR7nKXxUVtM,0.100000,"{parolayı, söyle}",{}
2,2,00:00:09.440,00:00:11.060,ya ben kim olacağım,JR7nKXxUVtM,25.100000,"{kim, ya, olacağım}",{ben}
3,3,00:00:11.060,00:00:11.860,keloğlan ım,JR7nKXxUVtM,0.100000,"{keloğlan, ım}",{}
4,4,00:00:13.380,00:00:13.880,ey,JR7nKXxUVtM,0.100000,{ey},{}
...,...,...,...,...,...,...,...,...
4995,4995,00:03:26.780,00:03:28.160,böyle bile yiyebiliriz,cQjX9DzXTw0,0.100000,"{bile, böyle, yiyebiliriz}",{}
4996,4996,00:03:28.260,00:03:30.540,kıvama geldiğinde artık tamamdır,cQjX9DzXTw0,0.100000,"{kıvama, artık, geldiğinde, tamamdır}",{}
4997,4997,00:03:30.540,00:03:34.840,böyle ağzınıza böyle çıtır çıtır şeker tadı da...,cQjX9DzXTw0,7.792308,"{böyle, önemli, tadı, gelmemesi, ağzınıza, şek...",{da}
4998,4998,00:03:34.840,00:03:37.060,şimdi bundan sonra artık çırpıcı ile devam ede...,cQjX9DzXTw0,0.100000,"{edeceğiz, sonra, çırpıcı, ile, artık, şimdi, ...",{}


In [59]:
df_sentence_ratio_result.loc[1,"intersect_word"] == '{}'

False

In [52]:
def dict_to_text(dict_value):
    list_of_dict = []
    for i in dict_value:
        if i == '':
            var = np.nan
        else:
            var = i
        list_of_dict.append(var)
    text = ", ".join(list_of_dict)

    return text

In [53]:
df_sentence_ratio_result["different_word"] = df_sentence_ratio_result["different_word"].apply(lambda x: dict_to_text(x))
df_sentence_ratio_result["intersect_word"] = df_sentence_ratio_result["intersect_word"].apply(lambda x: dict_to_text(x))
df_sentence_ratio_result 

,index,start_time,end_time,sentence,video_id,28_word_ratio,different_word,intersect_word
0,0,00:00:06.680,00:00:07.340,sen kimsin peki,JR7nKXxUVtM,33.433333,"kimsin, peki",sen
1,1,00:00:07.340,00:00:08.600,parolayı söyle,JR7nKXxUVtM,0.100000,"parolayı, söyle",
2,2,00:00:09.440,00:00:11.060,ya ben kim olacağım,JR7nKXxUVtM,25.100000,"kim, ya, olacağım",ben
3,3,00:00:11.060,00:00:11.860,keloğlan ım,JR7nKXxUVtM,0.100000,"keloğlan, ım",
4,4,00:00:13.380,00:00:13.880,ey,JR7nKXxUVtM,0.100000,ey,
...,...,...,...,...,...,...,...,...
4995,4995,00:03:26.780,00:03:28.160,böyle bile yiyebiliriz,cQjX9DzXTw0,0.100000,"bile, böyle, yiyebiliriz",
4996,4996,00:03:28.260,00:03:30.540,kıvama geldiğinde artık tamamdır,cQjX9DzXTw0,0.100000,"kıvama, artık, geldiğinde, tamamdır",
4997,4997,00:03:30.540,00:03:34.840,böyle ağzınıza böyle çıtır çıtır şeker tadı da...,cQjX9DzXTw0,7.792308,"böyle, önemli, tadı, gelmemesi, ağzınıza, şeke...",da
4998,4998,00:03:34.840,00:03:37.060,şimdi bundan sonra artık çırpıcı ile devam ede...,cQjX9DzXTw0,0.100000,"edeceğiz, sonra, çırpıcı, ile, artık, şimdi, d...",


In [54]:
df_sentence_ratio_result.loc[1,"intersect_word"]

''

In [9]:
#df_sentence_ratio_result.to_excel(f"Youtube_Sentence_{word_num}_Word_Ratio_Result.xlsx", index=False)

In [11]:
df_sentence_ratio_result = pd.read_excel(f"Youtube_Sentence_{word_num}_Word_Ratio_Result.xlsx")
df_sentence_ratio_result

,index,start_time,end_time,sentence,video_id,28_word_ratio,different_word,intersect_word
0,0,00:00:06.680,00:00:07.340,sen kimsin peki,JR7nKXxUVtM,33.433333,"kimsin, peki",sen
1,1,00:00:07.340,00:00:08.600,parolayı söyle,JR7nKXxUVtM,0.100000,"parolayı, söyle",NaN
2,2,00:00:09.440,00:00:11.060,ya ben kim olacağım,JR7nKXxUVtM,25.100000,"kim, ya, olacağım",ben
3,3,00:00:11.060,00:00:11.860,keloğlan ım,JR7nKXxUVtM,0.100000,"keloğlan, ım",NaN
4,4,00:00:13.380,00:00:13.880,ey,JR7nKXxUVtM,0.100000,ey,NaN
...,...,...,...,...,...,...,...,...
4995,4995,00:03:26.780,00:03:28.160,böyle bile yiyebiliriz,cQjX9DzXTw0,0.100000,"bile, böyle, yiyebiliriz",NaN
4996,4996,00:03:28.260,00:03:30.540,kıvama geldiğinde artık tamamdır,cQjX9DzXTw0,0.100000,"kıvama, artık, geldiğinde, tamamdır",NaN
4997,4997,00:03:30.540,00:03:34.840,böyle ağzınıza böyle çıtır çıtır şeker tadı da...,cQjX9DzXTw0,7.792308,"böyle, önemli, tadı, gelmemesi, ağzınıza, şeke...",da
4998,4998,00:03:34.840,00:03:37.060,şimdi bundan sonra artık çırpıcı ile devam ede...,cQjX9DzXTw0,0.100000,"edeceğiz, sonra, çırpıcı, ile, artık, şimdi, d...",NaN


#### Calculate Videoid Word Ratio

In [ ]:
#df_adjust_word = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{folder_name.lower().capitalize()}/Deployment/Result/Level 1/Opus/Word_{word_num}.xlsx")
#df_adjust_word

In [ ]:
#df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{folder_name.lower().capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
#df_youtube_sentence = df_youtube_sentence.head(5000)
#df_youtube_sentence

In [10]:
df_videoid_sentence = df_youtube_sentence.groupby("video_id")["sentence"].apply(" ".join).reset_index()
df_videoid_sentence

,video_id,sentence
0,23c9nwRm01o,arkadaşlar merhaba uzun zamandır video çekmiyo...
1,3jq75SKokNQ,asi döner alabilir miyim buyrun ne kadar hızlı...
2,4Ncsac3ywv0,intro çalıyor aman of of of of gidiyom gidemiy...
3,8V9tq1pe8eI,özgür bunlar normalde kamyon daha büyük araçla...
4,96Fw6H0ZG84,hiçbir şey yapmayacağım damlalarını vereceğim ...
5,CU7tcdhf02I,kar yağar kar üstüne derdim var dert üstüne ce...
6,DKTAB8HgIwc,dam başında sarı çiçek oy oy dam başında sarı ...
7,DgtKLrT9_GI,behey ala gözlü dilber dilber vaktin geçer dem...
8,DjrAJH4bjpE,eser rüzgar dağıtmaz efkarımı ben yarından umu...
9,EttVyq4DHYk,ve keten tohumu jeli ve kremi yapıyoruz sarkma...


In [65]:
def videoid_word_ratio(df_word, df_videoid_sentence, word_column_name, videoid_sentence_column_name, word_num):
    word_set = set(df_word[f"{word_column_name}"].to_list())
    for i in range(len(df_videoid_sentence)):
        text = df_videoid_sentence.loc[i,f"{videoid_sentence_column_name}"]
        text_word = re.findall(r"\w+", text, re.UNICODE)
        text_word_set = set(text_word)
        intersect_word = word_set.intersection(text_word_set)
        different_word = text_word_set.difference(word_set)
        df_videoid_sentence.loc[i,f"{word_num}_word_ratio"] = (len(intersect_word)/len(text_word)+0.001)*100
        df_videoid_sentence.loc[i,"different_word"] = [different_word]
        df_videoid_sentence.loc[i,"intersect_word"] = [intersect_word]
        
    return df_videoid_sentence

In [66]:
df_text_ratio_result = videoid_word_ratio(df_adjust_word, df_videoid_sentence, "word", "sentence", 28)
df_text_ratio_result

,video_id,sentence,28_word_ratio,different_word,intersect_word
0,23c9nwRm01o,arkadaşlar merhaba uzun zamandır video çekmiyo...,6.160606,"{arkadaşlar, aracın, öncelikle, aracı, kaplana...","{bir, çok, bu, var, ama, ve}"
1,3jq75SKokNQ,asi döner alabilir miyim buyrun ne kadar hızlı...,5.933333,"{ol, git, valla, boyu, kardeşim, getiriyorum, ...","{de, bir, kadar, bu, ben, var, ne}"
2,4Ncsac3ywv0,intro çalıyor aman of of of of gidiyom gidemiy...,4.751163,"{melodi, amman, gidiyom, aman, söyle, kalmaz, ...","{bana, bu, ben, ama, sen, ve}"
3,8V9tq1pe8eI,özgür bunlar normalde kamyon daha büyük araçla...,1.562317,"{devamı, durmuşum, yarı, kaldıracağım, güzelli...","{çok, ben, var, ama, sen, ve, bunu, beni, şey,..."
4,96Fw6H0ZG84,hiçbir şey yapmayacağım damlalarını vereceğim ...,3.983495,"{veterinerden, bakımını, şeylerin, özgürsün, ö...","{çok, var, ama, sen, ve, bunu, beni, şey, için..."
5,CU7tcdhf02I,kar yağar kar üstüne derdim var dert üstüne ce...,4.645455,"{amman, zaman, derdim, buğday, vursa, kar, boy...","{var, seni}"
6,DKTAB8HgIwc,dam başında sarı çiçek oy oy dam başında sarı ...,1.549275,"{göçek, feride, sarı, düşte, gidiyorum, kesek,...","{de, bir}"
7,DgtKLrT9_GI,behey ala gözlü dilber dilber vaktin geçer dem...,0.637634,"{gözlerin, behey, beller, olmuş, gözlü, harami...",{mi}
8,DjrAJH4bjpE,eser rüzgar dağıtmaz efkarımı ben yarından umu...,3.325806,"{verip, kendime, acıyı, eser, kalırın, edip, y...","{ben, mı, ne, için}"
9,EttVyq4DHYk,ve keten tohumu jeli ve kremi yapıyoruz sarkma...,3.098236,"{kaplarını, lekelerden, kullanırsınız, sürmeni...","{de, için, iyi, bir, da, çok, bu, yok, ben, da..."


#### Select Adjust Ratio Result

In [21]:
#df_sentence_ratio_result = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{folder_name}/Deployment/Result/Level 1/Youtube/Youtube_Sentence_28_Word_Ratio_Result.xlsx")
#df_sentence_ratio_result

In [32]:
adjust_word_ratio = 100
df_adjust_ratio = df_sentence_ratio_result[df_sentence_ratio_result[f"{word_num}_word_ratio"] >= adjust_word_ratio]
df_adjust_ratio

,index,start_time,end_time,sentence,video_id,28_word_ratio,different_word,intersect_word
59,59,00:00:15.780,00:00:16.600,ne,3jq75SKokNQ,100.1,{},{ne}
162,162,00:00:20.980,00:00:21.480,var,tCadoXCM31Y,100.1,{},{var}
164,164,00:00:23.280,00:00:24.060,var,tCadoXCM31Y,100.1,{},{var}
342,342,00:04:18.495,00:04:19.495,yok,UDDfBEUGY60,100.1,{},{yok}
350,350,00:04:45.747,00:04:46.747,ne,UDDfBEUGY60,100.1,{},{ne}
351,351,00:04:47.900,00:04:48.900,ne,UDDfBEUGY60,100.1,{},{ne}
400,400,00:07:35.246,00:07:36.278,değil mi evet,UDDfBEUGY60,100.1,{},"{mi, evet, değil}"
419,419,00:09:15.573,00:09:17.263,ben de çok,UDDfBEUGY60,100.1,{},"{çok, de, ben}"
549,549,00:21:32.301,00:21:33.545,sen de,UDDfBEUGY60,100.1,{},"{sen, de}"
564,564,00:23:33.434,00:23:34.434,evet,UDDfBEUGY60,100.1,{},{evet}


In [26]:
#df_adjust_ratio.to_excel(f"Youtube_Sentence_{word_num}_Word_{word_ratio}_Ratio.xlsx", index=False)

In [33]:
def dict_to_list_df(df_target, target_column): # df_target include dict value columns, target_column occurs dict
    dict_value_list = []
    for i in df_target[f"{target_column}"]:
        for j in i:
            dict_value_list.append(j)
            dict_value_unique = set(dict_value_list)
            df_unique_value = pd.DataFrame(dict_value_unique)
            df_unique_value.rename(columns={0:f"{target_column}_unique"}, inplace=True)
    return df_unique_value

In [34]:
df_unique_word = dict_to_list_df(df_adjust_ratio, "intersect_word")
df_unique_word

,intersect_word_unique
0,çok
1,iyi
2,de
3,sen
4,ve
5,seni
6,yok
7,mi
8,ama
9,var


#### Calculate Sequential Sentence Length Of Adjust Ratio Result

In [15]:
# From Youtube Sentence With Adjust Word

#### Count Adjust Ratio Result By Videoid

In [13]:
df_ratio_count = pd.DataFrame(df_adjust_ratio.groupby("video_id")["28_word_ratio"].count())
df_ratio_count.reset_index(inplace=True)
df_ratio_count.rename(columns={"28_word_ratio":"28_word_ratio_count"}, inplace=True)
df_ratio_count.sort_values(by="28_word_ratio_count", ascending=False, inplace=True)
df_ratio_count.reset_index(drop=True,inplace=True)
df_ratio_count

,video_id,28_word_ratio_count
0,xHyS_6Hq5GA,58
1,eA6FwMDpvfs,34
2,Aii8sJ5Uw_8,26
3,UDDfBEUGY60,26
4,YEndeSHpyBs,24
...,...,...
1021,Gvn1FneQG7Y,1
1022,GtOBr_mp59U,1
1023,aD9dEmSc5hs,1
1024,GqMXRktZraw,1


In [14]:
#df_ratio_count.to_excel(f"Youtube_Sentence_{word_num}_Word_{word_ratio}_Ratio_Videoid_Count.xlsx", index=False)